In [2]:
#import lib
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

# CONNECT TO WEB

In [3]:
#Connect and query HTML from website
URL = "https://vi.wikipedia.org/wiki/Danh_s%C3%A1ch_c%C3%B4ng_ty_l%E1%BB%9Bn_nh%E1%BA%A5t_th%E1%BA%BF_gi%E1%BB%9Bi_theo_doanh_thu"
HTML = requests.get(URL)
soup = BeautifulSoup(HTML.text, "html.parser" )

## Get columns name

In [4]:
#Location and get col name
table1 = soup.find("div", class_="mw-parser-output").find("table").find("tbody")
header_row = []
for i in table1.find('tr').find_all('th'):
    header_row.append(i.text.strip())
print(header_row) 

['Hạng', 'Tên', 'Ngành', 'Doanh thu(USD million)', 'Tốc độ tăng trưởng', 'Số nhân viên', 'Quốc gia', 'Chú thích']


In [5]:
#create dataframe by col name
head_col = pd.DataFrame(columns=header_row)
head_col

,Hạng,Tên,Ngành,Doanh thu(USD million),Tốc độ tăng trưởng,Số nhân viên,Quốc gia,Chú thích


In [6]:
## Location and Drop Dirty data
for err in table1.find_all('span', attrs={'style':'display:none'}):
    err.decompose()

In [7]:
values_table=table1.find_all('tr')[1:]

In [7]:
#get data from table
rows=[]
data=[]
for row in values_table:
    rows.append([n.text.strip() for n in row])
for el in rows:
    data.append(el[1::2])

### When I retrieved the data, I realized one thing some rows in the table were missing 1 - 2 data cells
### Then I researched and found out that the two missing data were 'Ngành' and 'Quốc gia'.

In [20]:
#filter any row missing data
miss=[]
for i in data:
    if len(i)<8:
        miss.append(i)

In [19]:
# Handling missing row of columns 'Ngành' with NaN
pat =re.compile('[0-9]+')
missing_industry=[]
for i in miss:
    if re.search(pat,i[2]):
        missing_industry.append(i)
for el in missing_industry:
    index_bf_missing = el.index(el[1])
    el.insert(index_bf_missing + 1,'NaN')

In [18]:
# Handling missing row of columns 'Quốc gia' with NaN
missing_country=[]
for i in miss:
    if re.search(pat,i[-2]):
        missing_country.append(i)
for el in missing_country:
    el.insert(el.index(el[-1]),'NaN')

In [11]:
data_t = data #backup

In [12]:
#check missing values again
for i in data_t:
    if len(i)<8:
        print(i)

In [13]:
#load_data to dataframe
df = head_col

for i in data_t:
    df.loc[len(df)]=i

In [15]:
#fill NaN with values above
for i in df['Ngành']:
    index_NaN = df.loc[df['Ngành']=='NaN'].index.tolist()
for ind in index_NaN:
    df.at[ind,'Ngành'] = df.at[ind-1,'Ngành']
    
for i in df['Quốc gia']:
    index_NaN = df.loc[df['Quốc gia']=='NaN'].index.tolist()
for ind in index_NaN:
    df.at[ind,'Quốc gia'] = df.at[ind-1,'Quốc gia']

In [16]:
df['Quốc gia'] = df['Quốc gia'].replace('China','Trung Quốc')

In [17]:
df

,Hạng,Tên,Ngành,Doanh thu(USD million),Tốc độ tăng trưởng,Số nhân viên,Quốc gia,Chú thích
0,1,Walmart,Retail,"$514,405",2.8%,"2,200,000",Hoa Kỳ,[4]
1,2,Sinopec,Oil and gas,"$414,649",26.8%,"619,151",Trung Quốc,[5]
2,3,Royal Dutch Shell,Oil and gas,"$396,556",27.2%,"81,000",Hà Lan,[6]
3,4,China National Petroleum,Oil and gas,"$392,976",20.5%,"1,382,401",Trung Quốc,[7]
4,5,State Grid,Electricity,"$387,056",10.9%,"917,717",Trung Quốc,[8]
5,6,Saudi Aramco,Oil and gas,"$355,905",35.3%,"76,418",Ả Rập Saudi,[9]
6,7,BP,Oil and gas,"$303,738",24.2%,"73,000",United Kingdom,[10]
7,8,ExxonMobil,Oil and gas,"$290,212",18.8%,"71,000",United States,[11]
8,9,Volkswagen,Automotive,"$278,341",7.0%,"664,496",Đức,[12]
9,10,Toyota,Automotive,"$272,612",2.8%,"370,870",Nhật Bản,[13]
